### # Instalando dependências

No código abaixo, serão instalados as seguintes dependências:
1. [OpenPyXL](https://openpyxl.readthedocs.io/en/stable/)
2. [Selenium](https://www.selenium.dev/pt-br/documentation/webdriver/getting_started/)
3. [Docx2txt](https://github.com/ankushshah89/python-docx2txt)
4. [PyMuPDF](https://pymupdf.readthedocs.io/en/latest/module.html)

In [ ]:
%pip install -r requirements.txt

In [7]:
import os
import re
import time
import fitz
import docx2txt
import requests

from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from openpyxl import Workbook

### # Definindo parâmetros
Nesta seção você deverá definir o raio de boletins que você deseja pegar os dados e, portanto, adicionar na planilha. Na variável **min** e **max** representam o raio que você deseja buscar os boletins, ou seja, se definirmos **max** como *10* e **min** como *1*, estamos definindo um raio entre o boletim de número 1 até o boletim de número 10

In [ ]:
max = 0 # se 0, irá até o ultimo boletim. 
min = 50

### # Variáveis Estáticas

Nesta seção, estão definidas algumas variáveis importantes para o funcionamento da aplicação e não devem ser modificadas

In [9]:
path = os.path.join(os.getcwd(), 'downloads')
url = 'https://portal.tcu.gov.br/jurisprudencia/boletins-e-informativos/'

### # Métodos Importantes

In [ ]:
def save_file(context, path, extension):
    return

### # Configurando o Selenium

Nesta seção, o navegador que será utilizado para realizar web scraping será configurado e instanciado para realizar a atividade

In [11]:
options = ChromeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": path,
    "download.directory_upgrade": True,
    "download.prompt_for_download": False,
})

driver = Chrome(options)
wait = WebDriverWait(driver, 20)

### # Acessando o site do TCE

Neste processo, entraremos no site dos boletins de jurisprudência e coletaremos algumas informações como a quantidade de boletins e sobre a paginação da tabela dos boletins

In [13]:
driver.get(url)

In [1]:
class Judgment:
    def __init__(self, judgment, year, rapporteur, process, subject, type, header, body, session_date, url):
        self.judgment = judgment
        self.year = year
        self.rapporteur = rapporteur
        self.process = process
        self.subject = subject
        self.type = type
        self.header = header
        self.body = body
        self.session_date = session_date
        self.url = url
        self.push_content()
        
    def push_content(self):
        driver.get(self.url)     
        self.type = self.wait.until(EC.visibility_of_element_located((By.ID, 'conteudo_tipo_processo'))).text
        self.session_date = self.wait.until(EC.visibility_of_element_located((By.ID, 'conteudo_data_sessao'))).text
        self.process = self.wait.until(EC.visibility_of_element_located((By.ID, 'conteudo_processo'))).text.replace(' launch', '')
        self.rapporteur = self.wait.until(EC.visibility_of_element_located((By.ID, 'conteudo_relator'))).text
        self.subject = self.wait.until(EC.visibility_of_element_located((By.ID, 'conteudo_assunto'))).text

class Report:
    def __init__(self, report, release_date, url):
        self.report = report
        self.release_date = release_date
        self.url = url
        self.download()
        self.push()
        
    def download(self):
        pdf_response = requests.get(self.url)
        docx_response = requests.get(self.url.replace('&amp;', '&'))    
        
        if pdf_response.status_code != 200 or docx_response.status_code != 200:
            return
        
        save_file(docx_response.context, path, 'docx')
        save_file(pdf_response.context, path, 'pdf')
        
    def push_content(self):
        return